In [206]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, IntegerType, DateType, StringType
from pyspark.sql.functions import udf, isnan, when, count, col, coalesce, lit, avg, mean, concat, concat_ws

In [2]:
import os 
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/share/java/mariadb-java-client-2.5.3.jar pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /Users/majo/Documents/david/mariadb-java-client-2.5.3.jar pyspark-shell'

In [3]:
spark_context = SparkContext()
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

21/11/07 20:26:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/07 20:26:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/Users/majo/opt/anaconda3/envs/Modelo_ETL/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
PATH = "/Users/majo/Desktop/Maestria_MIAD/Semestre_1/Modelado_de_Datos_y_ETL/Semana_2/Tarea_Data_Profiling/"

In [16]:
aeropuertos = spark.read.load(PATH+"aeropuertos.csv",format="csv", sep=",", inferSchema="true", header="true")

In [17]:
print(aeropuertos.schema)

StructType(List(StructField(_c0,IntegerType,true),StructField(sigla,StringType,true),StructField(iata,StringType,true),StructField(nombre,StringType,true),StructField(municipio,StringType,true),StructField(departamento,StringType,true),StructField(categoria,StringType,true),StructField(latitud,DoubleType,true),StructField(longitud,DoubleType,true),StructField(propietario,StringType,true),StructField(explotador,StringType,true),StructField(longitud_pista,DoubleType,true),StructField(ancho_pista,DoubleType,true),StructField(pbmo,StringType,true),StructField(elevacion,DoubleType,true),StructField(resolucion,StringType,true),StructField(fecha_construccion,StringType,true),StructField(fecha_vigencia,StringType,true),StructField(clase,StringType,true),StructField(tipo,StringType,true),StructField(numero_vuelos_origen,StringType,true),StructField(gcd_departamento,IntegerType,true),StructField(gcd_municipio,IntegerType,true)))


In [18]:
row_number = aeropuertos.count()
print(row_number)

292


In [19]:
aeropuertos = aeropuertos.drop("_c0", "iata", "latitud", "longitud", "propietario", "explotador", "longitud_pista", "ancho_pista", "pbmo", "orientacion", "elevacion", "resolucion", "calse", "tipo", "numero_vuelos_origen", "gcd_departamento", "gcd_municipio", "fecha_vigencia")

In [20]:
aeropuertos.select([count(when(isnan(c), c)).alias(c) for c in aeropuertos.columns]).show()

+-----+------+---------+------------+---------+------------------+-----+
|sigla|nombre|municipio|departamento|categoria|fecha_construccion|clase|
+-----+------+---------+------------+---------+------------------+-----+
|    0|     0|        0|           0|        0|                 0|    0|
+-----+------+---------+------------+---------+------------------+-----+



In [27]:
aeropuertos = aeropuertos.withColumn("fecha_construccion", aeropuertos["fecha_construccion"].cast(DateType()))

In [29]:
print("sigla: ", aeropuertos.select("sigla").distinct().count())
print("nombre: ", aeropuertos.select("nombre").distinct().count())
print("municipio: ", aeropuertos.select("municipio").distinct().count())
print("departamento: ", aeropuertos.select("departamento").distinct().count())
print("categoria: ", aeropuertos.select("categoria").distinct().count())

sigla:  212
nombre:  206
municipio:  122
departamento:  30
categoria:  4


In [40]:
aeropuertos = aeropuertos.dropDuplicates()

In [41]:
aeropuertos = aeropuertos.where(aeropuertos['fecha_construccion']<'2014-01-01')

In [147]:
vuelos = spark.read.load(PATH+"vuelos.csv",format="csv", sep=",", inferSchema="true", header="true")

In [179]:
print(vuelos.schema)

StructType(List(StructField(ano,IntegerType,true),StructField(mes,IntegerType,true),StructField(origen,StringType,true),StructField(destino,StringType,true),StructField(tipo_vuelo,StringType,true),StructField(trafico,StringType,true),StructField(vuelos,IntegerType,true),StructField(sillas,IntegerType,true),StructField(carga_ofrecida,IntegerType,true),StructField(pasajeros,IntegerType,true),StructField(carga_bordo,IntegerType,true)))


In [149]:
row_number = vuelos.count()
print(row_number)

67599


In [150]:
vuelos = vuelos.drop("tipo_equipo", "empresa")

In [151]:
vuelos.select([count(when(isnan(c), c)).alias(c) for c in vuelos.columns]).show()

+---+---+------+-------+----------+-------+------+------+--------------+---------+-----------+
|ano|mes|origen|destino|tipo_vuelo|trafico|vuelos|sillas|carga_ofrecida|pasajeros|carga_bordo|
+---+---+------+-------+----------+-------+------+------+--------------+---------+-----------+
|  0|  0|     0|      0|         0|    894|  1582|  4017|             0|     3483|          0|
+---+---+------+-------+----------+-------+------+------+--------------+---------+-----------+



In [178]:
vuelos = vuelos.withColumn("vuelos",vuelos["vuelos"].cast(IntegerType()))
vuelos = vuelos.withColumn("sillas",vuelos["sillas"].cast(IntegerType()))
vuelos = vuelos.withColumn("pasajeros",vuelos["pasajeros"].cast(IntegerType()))
vuelos = vuelos.withColumn("carga_ofrecida",vuelos["carga_ofrecida"].cast(IntegerType()))
vuelos = vuelos.withColumn("carga_bordo",vuelos["carga_bordo"].cast(IntegerType()))
vuelos = vuelos.withColumn("trafico",vuelos["trafico"].cast(StringType()))

In [200]:
promedio_vuelos = vuelos.agg({'vuelos': 'avg'})
vuelos = vuelos.fillna({'vuelos': promedio_vuelos.first()[0]})

promedio_sillas = vuelos.agg({'sillas': 'avg'})
vuelos = vuelos.fillna({'sillas': promedio_sillas.first()[0]})

promedio_pasajeros = vuelos.agg({'pasajeros': 'avg'})
vuelos = vuelos.fillna({'pasajeros': promedio_pasajeros.first()[0]})

#vuelos = vuelos['trafico'].fillna(method = "backfill")

vuelos.show()

TypeError: 'Column' object is not callable

In [186]:
vuelos.select([count(when(isnan(c), c)).alias(c) for c in vuelos.columns]).show()
vuelos.where(col("trafico").isNull()).show()

+---+---+------+-------+----------+-------+------+------+--------------+---------+-----------+
|ano|mes|origen|destino|tipo_vuelo|trafico|vuelos|sillas|carga_ofrecida|pasajeros|carga_bordo|
+---+---+------+-------+----------+-------+------+------+--------------+---------+-----------+
|  0|  0|     0|      0|         0|    894|     0|     0|             0|        0|          0|
+---+---+------+-------+----------+-------+------+------+--------------+---------+-----------+

+---+---+------+-------+----------+-------+------+------+--------------+---------+-----------+
|ano|mes|origen|destino|tipo_vuelo|trafico|vuelos|sillas|carga_ofrecida|pasajeros|carga_bordo|
+---+---+------+-------+----------+-------+------+------+--------------+---------+-----------+
+---+---+------+-------+----------+-------+------+------+--------------+---------+-----------+



In [49]:
print("origen: ", vuelos.select("origen").distinct().count())
print("destino: ", vuelos.select("destino").distinct().count())
print("tipo_vuelo: ", vuelos.select("tipo_vuelo").distinct().count())
print("trafico: ", vuelos.select("trafico").distinct().count())

origen:  111
destino:  110
tipo_vuelo:  4
trafico:  2


In [84]:
vuelos = vuelos.dropDuplicates()

In [195]:
trayecto = vuelos[['origen', 'destino']]
trayecto = trayecto.select(concat(trayecto.origen, trayecto.destino).alias("idTrayecto"), "origen", "destino")
trayecto = trayecto.dropDuplicates()
trayecto.write.csv("trayecto.csv")

In [197]:
region = aeropuertos[['departamento', 'municipio']]
region = region.dropDuplicates()
region.write.csv("region.csv")

In [208]:
registro_vuelos = vuelos[['ano', 'mes', 'origen', 'destino', 'vuelos', 'pasajeros', 'sillas', 'tipo_vuelo', 'trafico', 'carga_ofrecida', 'carga_bordo']]
registro_vuelos = registro_vuelos.withColumn("dia", lit("01"))
registro_vuelos = registro_vuelos.select(concat_ws('-', registro_vuelos.ano, registro_vuelos.mes, registro_vuelos.dia)
                                         .cast("date")
                                         .alias("idFecha"),concat(registro_vuelos.origen, registro_vuelos.destino)
                                         .alias("idTrayecto"), 'vuelos', 'pasajeros', 'sillas', 'tipo_vuelo', 
                                         'trafico', 'carga_ofrecida', 'carga_bordo')

+----------+----------+------+---------+------+----------+-------+--------------+-----------+
|   idFecha|idTrayecto|vuelos|pasajeros|sillas|tipo_vuelo|trafico|carga_ofrecida|carga_bordo|
+----------+----------+------+---------+------+----------+-------+--------------+-----------+
|2012-01-01|    BOGCUC|     1|        4|     0|         T|      N|             0|        100|
|2013-05-01|    UIBBOG|    30|      873|  1110|         R|      N|         24000|       4222|
|2013-10-01|    IBEBOG|    98|     2866|  3626|         R|      N|         56056|       2323|
|2012-04-01|    FLABOG|     1|        4|     0|         T|      N|             0|          0|
|2013-07-01|    CUCAUC|     1|      911|     0|         T|      N|             0|        180|
|2012-03-01|    CUCOCV|     6|        6|     0|         T|      N|             0|          0|
|2013-11-01|    7NSBOG|    14|      149|  1133|         T|      N|             0|        880|
|2010-06-01|    AUCTME|     1|        2|     0|         T|  